# 情绪对股票市场的影响

## 1.数据准备
风险偏好
事件

In [1]:
import sys
import os
import numpy as np
import cudf  #CUDA计算
import pandas as pd

sys.path.append('/home/ubuntu/notebook/Investor-Sentiment')
sys.path.append('/usr/local/stata17/utilities')
from pystata import config  #Stata
from statsmodels.regression.rolling import RollingOLS  #滚动回归
from statsmodels.regression.linear_model import OLS  #OLS回归
from pandarallel import pandarallel  #多线程groupby Apply
import matplotlib.pyplot as plt

# 数据库
from utils.sql import DB
from loader.findata_loader import DownLoader

# Stata
config.init('mp')

# ------------------------------数据集路径----------------------------------#
DATASETS_PATH = '/data/DataSets/investor_sentiment/'


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Single-user 8-core , expiring  1 Jan 2025
Serial number: 501709301094
  Licensed to: Colin's Stata
               Love U

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


### 1.1 下载和合并面板数据

In [2]:
# 数据集:个股K线面板数据,个股基本面数据
def load_data(): DownLoader(MAX_CORE=10).load_data()


load_data()

0it [00:00, ?it/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]
0it [00:00, ?it/s]


其他时间序列数据源

#### 指数盈利预测面板数据

In [2]:
def get_conidx_panel() -> cudf.DataFrame:
    """
    返回指数一致预期面板数据
    :return: 面板数据
    """
    df = cudf.read_parquet(DATASETS_PATH + 'CON_FORECAST_IDX.parquet')
    # 筛选研究样本
    df['CON_DATE'] = df['CON_DATE'].dt.strftime('%Y%m%d')

    # 格式处理
    df = df.astype(dtype={'CON_DATE': 'uint32', 'CON_YEAR': 'uint32'})

    # 筛选样本期 短期预期 选择指数样本
    df = df[(df['CON_DATE'] >= 20131231) & ((df['CON_DATE'] // 10000) == df['CON_YEAR'])
            & ((df['INDEX_CODE'].str[:1] == '0') | (df['INDEX_CODE'].str[:1] == '3'))]

    # 改列名
    df['INDEX_CODE'] = (np.where(df['INDEX_CODE'].to_pandas().str[:1] == '0',
                                 df['INDEX_CODE'].to_pandas() + '.SH',
                                 df['INDEX_CODE'].to_pandas() + '.SZ'))

    # 方便合并
    df = df.rename(columns={'CON_DATE': 'trade_date', 'INDEX_CODE': 'ts_code'}).set_index(['trade_date', 'ts_code'])

    # 删掉不要的
    return df.drop(columns=['index', 'ID', 'ENTRYTIME', 'ENTRYTIME', 'UPDATETIME', 'TMSTAMP', 'INDEX_NAME'])


get_conidx_panel()

CON_YEAR        CON_OR        CON_NP  CON_EPS  \
trade_date ts_code                                                    
20131231   000002.SH      2013  2.150299e+09  1.996171e+08   0.6183   
           000010.SH      2013  1.616497e+09  1.803029e+08   0.7227   
           000015.SH      2013  9.223393e+08  1.402562e+08   0.7359   
           000016.SH      2013  1.095701e+09  1.190502e+08   0.7747   
           000067.SH      2013  1.304862e+08  7.875350e+06   0.5807   
...                        ...           ...           ...      ...   
20221028   399312.SZ      2022  3.583564e+09  3.023053e+08   1.1050   
           399313.SZ      2022  2.848629e+09  3.164494e+08   1.1574   
           399317.SZ      2022  7.321688e+09  5.934493e+08   0.7403   
           000985.SH      2022  7.224144e+09  5.840384e+08   0.7555   
           000688.SH      2022  6.009689e+07  7.351126e+06   1.0083   

                            CON_NA  CON_PB  CON_PS   CON_PE  CON_PEG  CON_ROE  \
trade_date ts_code                                                              
20131231   000002.SH  1.407965e+09  1.2913  0.8455   9.1078   0.5689    14.18   
           000010.SH  1.156060e+09  1.1894  0.8506   7.6264   0.5842    15.60   
           000015.SH  8.384264e+08  1.0141  0.9219   6.0624   0.5667    16.73   
           000016.SH  7.548906e+08  1.1679  0.8046   7.4054   0.5799    15.77   
           000067.SH  6.214941e+07  2.2588  1.0758  17.8255   0.9922    12.67   
...                            ...     ...     ...      ...      ...      ...   
20221028   399312.SZ  2.454648e+09  1.4467  0.9909  11.7465   0.7619    12.32   
           399313.SZ  2.754211e+09  0.9901  0.9572   8.6169   0.7773    11.49   
           399317.SZ  5.905421e+09  1.2950  1.0445  12.8868   0.7655    10.05   
           000985.SH  5.825681e+09  1.3191  1.0637  13.1574   0.7802    10.03   
           000688.SH  6.128843e+07  4.1037  4.1850  34.2135   0.9417    11.99   

                      CON_OR_YOY  CON_NP_YOY  CON_NPCGRATE_2Y  CON_PEI  
trade_date ts_code                                                      
20131231   000002.SH        9.89       18.25          16.0105  78.4365  
           000010.SH        8.21       13.78          13.0548  80.9011  
           000015.SH        5.53       11.24          10.6985  83.8420  
           000016.SH        7.24       13.37          12.7691  81.7573  
           000067.SH       12.46       16.76          17.9659  61.4964  
...                          ...         ...              ...      ...  
20221028   399312.SZ       15.04       15.51          15.4183  94.5570  
           399313.SZ       14.36       10.80          11.0863  94.4014  
           399317.SZ       10.82       15.55          16.8354  80.5216  
           000985.SH       10.96       15.11          16.8637  80.1858  
           000688.SH       43.95       46.41          36.3298  98.5569  

[43510 rows x 14 columns]

#### A股基本面面板数据

In [3]:
def get_ashare_panel() -> cudf.DataFrame:
    """
    返回A股面板数据 需要基本面的其他指标增加columns即可
    :return:
    """
    df = (
            cudf.concat(
                    [
                            # A股K线数据
                            cudf.read_parquet(f'{DATASETS_PATH}ASHARE_BAR_PANEL.parquet', columns=['trade_date', 'ts_code', 'pct_chg'])
                            .rename(columns={'pct_chg': 'share_return'}),
                            # A股基本面数据
                            cudf.read_parquet(f'{DATASETS_PATH}ASHARE_BASIC_PANEL.parquet',
                                              columns=['trade_date', 'ts_code', 'total_mv'])
                    ],
                    join="left", axis=1, sort=True
            ).query('trade_date>=20140101')

    )
    return df


get_ashare_panel()

share_return     total_mv
trade_date ts_code                             
20140102   000001.SZ       -0.1641  10025372.09
           000002.SZ       -0.4972   8799966.25
           000004.SZ        1.3734   99176.4638
           000005.SZ       -0.4000  227669.0681
           000006.SZ       -1.2164  657447.5874
...                            ...          ...
20221130   872374.BJ       -1.7259         <NA>
20221201   301290.SZ       -8.7349  485640.3232
           301311.SZ       12.7436     541440.0
           870199.BJ       -3.1447     149842.0
           872374.BJ        0.2066         <NA>

[7143120 rows x 2 columns]

#### 指数基本面面板数据

In [4]:
def get_index_panel() -> cudf.DataFrame:
    """
    返回指数面板数据 需要基本面的其他指标增加columns即可
    :return:
    """
    df = (
            cudf.concat(
                    [
                            # 指数K线数据
                            cudf.read_parquet(f'{DATASETS_PATH}IDX_BAR_PANEL.parquet', columns=['trade_date', 'ts_code', 'pct_chg'])
                            .rename(columns={'pct_chg': 'shareindex_return'}),
                            # 指数基本面数据
                            cudf.read_parquet(f'{DATASETS_PATH}IDX_BASIC_PANEL.parquet',
                                              columns=['trade_date', 'ts_code', 'total_mv', 'total_share', 'pe', 'pe_ttm']),
                    ],
                    join="left", axis=1, sort=True
            ).query('trade_date>=20140101')

    )
    return df


get_index_panel()

shareindex_return      total_mv   total_share     pe  \
trade_date ts_code                                                          
20140102   000001.SH           -0.3115  1.816591e+13  3.233397e+12  10.71   
           000016.SH           -0.8688  8.764339e+12  1.536462e+12   8.35   
           000300.SH           -0.3454  1.724706e+13  2.880502e+12   9.79   
           000905.SH            0.4905  3.684081e+12  4.173544e+11  31.71   
           399001.SZ           -0.0911  1.809932e+12  1.431701e+11  16.19   
...                                ...           ...           ...    ...   
20221207   000016.SH           -0.5288  1.798471e+13  1.586560e+12  10.08   
           000300.SH           -0.2459  4.800432e+13  4.012892e+12  12.06   
           000905.SH           -0.1167  1.237585e+13  1.360448e+12  20.31   
           399001.SZ             0.175  2.251881e+13  1.292449e+12  25.40   
           399300.SZ              <NA>  4.800432e+13  4.012892e+12  12.06   

                      pe_ttm  
trade_date ts_code            
20140102   000001.SH    9.64  
           000016.SH    7.51  
           000300.SH    8.89  
           000905.SH   28.78  
           399001.SZ   14.84  
...                      ...  
20221207   000016.SH    9.57  
           000300.SH   11.55  
           000905.SH   23.74  
           399001.SZ   26.56  
           399300.SZ   11.55  

[13050 rows x 5 columns]

#### 时间序列数据

In [5]:
def get_time_series() -> cudf.DataFrame:
    """
    返回情绪,无风险收益
    :return:
    """
    df = (cudf.concat(
            [
                    # 情绪数据
                    cudf.from_pandas(
                            pd.concat(
                                    [pd.read_sql_table('IMG_SENT', DB().ENGINE, 'SENT_DAILY').astype(dtype={'trade_date': 'uint32'})
                                     .set_index('trade_date').rename(columns={'neg_index': 'img_neg'}),

                                     pd.read_sql_table('TEX_SENT', DB().ENGINE, 'SENT_DAILY').astype(dtype={'trade_date': 'uint32'})
                                     .set_index('trade_date').rename(columns={'neg_index': 'tex_neg'})
                                     ], axis=1
                            )),
                    # SHIBOR数据
                    cudf.from_pandas((pd.read_sql_table('SHIBOR', DB().ENGINE, 'FIN_DAILY_MACRO', columns=['trade_date', '3m'])
                                      .astype(dtype={'trade_date': 'uint32'}).set_index('trade_date')
                                      .rename(columns={'3m': 'riskfree_return'}) / 360))
            ],
            axis=1, sort=True))

    return df


get_time_series()

,img_neg,tex_neg,riskfree_return
trade_date,,,
20140102,0.0,0.0,0.015460
20140103,0.0,0.333333,0.015461
20140106,0.285714,0.142857,0.015481
20140107,0.0,0.333333,0.015489
20140108,0.333333,0.0,0.015506
...,...,...,...
20221125,<NA>,<NA>,0.006078
20221128,<NA>,<NA>,0.006069
20221129,<NA>,<NA>,0.006072


## 2.多群体视角下的情绪分析

先求出Profit,再用每日的变动解释,
先用000300.SH

In [6]:
df_te2 = cudf.read_parquet('/data/DataSets/investor_sentiment/IDX_PANEL_PROFIT.parquet').to_pandas().query("ts_code=='000300.SH'")
df_test = get_index_panel().to_pandas().query("ts_code=='000300.SH'")

In [ ]:
# 求实际eps和预期eps
df_index_panel = get_index_panel()
# df_index_panel['eps_ttm'] = (df_index_panel['total_mv'] / df_index_panel['total_share']) / df_index_panel['pe_ttm']
df_index_panel = df_index_panel[['pe_ttm', 'pe']]
# df_index_panel

# get_conidx_panel().to_pandas().query("ts_code=='000300.SZ'")
# pd.concat([df_index_panel.to_pandas(),
#            get_conidx_panel().to_pandas()], axis=1
#           )
df_temp = cudf.merge(df_index_panel, get_conidx_panel(), on=['trade_date', 'ts_code'], how='left', sort=True).dropna(
        axis=0).to_pandas()
# df_temp[''] = df_temp['CON_PE'] - df_temp['pe_ttm']
df_temp
# @formatter:off

In [ ]:
%%stata -d df_temp -force
reg pe_ttm CON_PE
# @formatter:on

## 3.截面效应分析

#### 3.1 计算面板数据的异质波动率

提取用于回归的数据

In [ ]:
df_panel = (
        cudf.merge(
                # 增加用于回归的无风险利率
                cudf.merge(get_ashare_panel(), get_time_series()[['riskfree_return']], on='trade_date', how='left', sort=True),
                # 增加用于回归的市场指数
                cudf.from_pandas(
                        get_index_panel()[['shareindex_return']]
                        .to_pandas().query("ts_code == '000300.SH'").reset_index('ts_code', drop=True)
                ),
                on='trade_date', how='left', sort=True
        ).to_pandas()
)
df_panel

In [ ]:
# 滚动OLS回归求异质波动率
def roll_idvol(df_code: pd.DataFrame, ols_window: int, var_ma: int) -> pd.DataFrame:
    try:
        # 估计参数
        model_ols = RollingOLS(endog=df_code[['Y']], exog=df_code[['CONST', 'X']], window=ols_window)
        df_para = model_ols.fit().params.rename(columns={'CONST': 'Alpha', 'X': 'Beta'})

        # 预测残差 已经对齐了
        df_con = pd.concat([df_code, df_para], axis=1, join='inner')
        df_con['Residual'] = df_con['Alpha'] + df_con['Beta'] * df_con['X'] - df_con['Y']

        # 计算月波动率
        df_con['Idvol'] = df_con['Residual'].rolling(var_ma).var(ddof=1)
        return df_con[['share_return', 'total_mv', 'Idvol']]
    except (IndexError, ValueError): return pd.DataFrame(columns=['trade_date', 'ts_code']).set_index(['trade_date', 'ts_code'])


# 分组计算
def cal_panel_ols():
    # 定义回归变量 CAPM回归: (rm-rf)=a+b*(RM-rf)
    df_panel['Y'] = df_panel['share_return'] - df_panel['riskfree_return']
    df_panel['CONST'] = 1  # 带截距项回归
    df_panel['X'] = df_panel['shareindex_return'] - df_panel['riskfree_return']

    # 多线程加速
    pandarallel.initialize(progress_bar=True)
    df_out = (df_panel.groupby(level=['ts_code'])[['share_return', 'total_mv', 'Y', 'CONST', 'X']]
              .parallel_apply(lambda x: roll_idvol(x, 5, 30)).droplevel(2)
              )

    # 保存
    df_out.to_parquet(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet', engine='pyarrow', index=True)


# 计算滚动回归
if not os.path.exists(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet'): cal_panel_ols()

# 加载滚动回归的面板数据集
df_ols_panel = (
        cudf.read_parquet(f'{DATASETS_PATH}ASHARE_OLS_PANEL.parquet').reset_index().set_index(['trade_date', 'ts_code']).sort_index()
        .rename(columns={'Idvol': 'idvol'})
)
df_ols_panel

### 3.2 面板数据分组

In [ ]:
# 参数阈值
QUANTILE = 0.5


# 面板数据分组
def group_ols_panel(df, group_col: str):
    """
    :param df:
    :param group_col:分组变量
    """
    # 分组
    df[f'{group_col}_split'] = df[group_col].groupby(level=['trade_date']).transform(lambda x: x.quantile(QUANTILE))
    df[f'{group_col}_group'] = np.where(df[group_col].to_pandas() >= df[f'{group_col}_split'].to_pandas(),
                                        f'{group_col}_high', f'{group_col}_low')
    # 重新设定分组索引
    df = df.reset_index().set_index(['trade_date', f'{group_col}_group', 'ts_code']).sort_index()

    # 求组中市值加权系数
    df[f'{group_col}_mv_ratio'] = df['total_mv'] / df.groupby(level=['trade_date', f'{group_col}_group'])['total_mv'].transform('sum')

    # 求组中回报
    df[f'{group_col}_vw_return'] = df[f'{group_col}_mv_ratio'] * df['share_return']
    df[f'{group_col}_group_return'] = (df.groupby(level=['trade_date', f'{group_col}_group'])[f'{group_col}_vw_return']
                                       .transform('sum'))

    # 重置索引
    return df.reset_index(f'{group_col}_group')


# 循环分组,最后处理成时间序列数据
def group_cols(df, columns: list):
    # 用于分组
    df_temp = df

    # 求组中市值加权回报
    for col in columns: df_temp = group_ols_panel(df_temp, col)

    # 筛选
    df_temp = df_temp[[i + '_group' for i in columns] + [i + '_group_return' for i in columns]]

    # 保留唯一的组合 2^N
    df_time_panel = (df_temp.reset_index().groupby(['trade_date'] + [i + '_group' for i in columns]).first())

    # 转为时间序列数据
    df_time = cudf.DataFrame()
    for col in columns: df_time = cudf.concat([df_time,
                                               df_time_panel.groupby(level=['trade_date', f'{col}_group']).first().reset_index()
                                              .pivot(index='trade_date', columns=f'{col}_group', values=f'{col}_group_return')],
                                              join="left", axis=1, sort=True)
    # 高减低
    # for col in columns: df_time[f'{col}_mid'] = df_time[f'{col}_high'] - df_time[f'{col}_low']

    return df_time


df_group_time = group_cols(df_ols_panel, ['total_mv', 'idvol'])
df_group_time

## 4. VAR模型分析

### 4.1 回归前数据处理

In [ ]:
# 增加用于回归的市场指数
df_time_join = (
        cudf.concat([get_time_series().drop(columns='riskfree_return'), df_group_time,
                     # 指数数据
                     cudf.from_pandas(get_index_panel()[['shareindex_return']].to_pandas().query("ts_code == '000001.SH'")
                                      .reset_index('ts_code', drop=True).rename(columns={'shareindex_return': 'idx_000001'})),
                     # 指数数据
                     cudf.from_pandas(get_index_panel()[['shareindex_return']].to_pandas().query("ts_code == '000300.SH'")
                                      .reset_index('ts_code', drop=True).rename(columns={'shareindex_return': 'idx_000300'})),
                     # 指数数据
                     cudf.from_pandas(get_index_panel()[['shareindex_return']].to_pandas().query("ts_code == '000016.SH'")
                                      .reset_index('ts_code', drop=True).rename(columns={'shareindex_return': 'idx_000016'})),
                     # 指数数据
                     cudf.from_pandas(get_index_panel()[['shareindex_return']].to_pandas().query("ts_code == '399300.SZ'")
                                      .reset_index('ts_code', drop=True).rename(columns={'shareindex_return': 'idx_399300'})),
                     ], sort=True, axis=1
                    ).dropna(axis=0).to_pandas()
)
df_time_join

In [ ]:
# 增加平方项
def add_square_column(df): return pd.concat([df, df.pow(2).add_suffix('_s')], axis=1)


# 增加日期虚拟变量
def add_dummy_column(df, column: str):
    df_weekday = pd.get_dummies(pd.to_datetime(df[column], format='%Y%m%d').dt.weekday, prefix='weekday', drop_first=True)
    df_month = pd.get_dummies(pd.to_datetime(df[column], format='%Y%m%d').dt.month, prefix='month', drop_first=True)
    return pd.concat([df, df_weekday, df_month], axis=1)


# 处理好的用于回归的数据
df_series_ols = add_dummy_column(add_square_column(df_time_join).reset_index(), 'trade_date')
df_series_ols
# @formatter:off

### 4.2 回归结果

In [ ]:
%%stata -d df_series_ols -force
//描述性统计
logout, save(Outputs/Table_Sum)  replace: ///
tabstat *_neg idx_* *_high *_low, s(N sd mean p50 min max ) f(%12.4f) c(s)

#### 4.2.1 主要股票市场

In [ ]:
%%stata -d df_series_ols -force -nogr
//时间设定
ge time = _n
tsset time
est clear

//VAR回归
foreach var in idx_000001 idx_000300 idx_000016 idx_399300    {
    rename(`var' `var'_s) (return return_s)

    eststo: qui var return img_neg return_s, lags(1/5) exog(month_* weekday_*)
    estadd local Month "Yes", replace
    estadd local Weekday "Yes", replace

    //绘图
    irf creat var, set(Outputs/`var'_img ,replace) step(5)
    irf graph oirf, impulse(img_neg) response(return) lstep(0) ustep(5) name(`var'_img,replace)  ///
    byopts(note("")) byopts(legend(off)) xtitle(, size(small) margin(zero)) ///
    ysc(r(-0.15,0.15)) yline(0) ylabel(#2) ytitle(return, size(small) margin(zero)) scheme(sj)

    rename(return return_s) (`var' `var'_s)
}

//输出
esttab , keep(return:L*.img_neg) ///
star(* 0.1 ** 0.05 *** 0.01) ///
stats( Month Weekday  r2_1 N, fmt(%3s %3s %12.4f %12.0f)) b(%12.4f) ///
title("Table1 Main Market") mtitle("000001.SH" "000300.SH" "000016.SH" "399300.SZ")  nogap

脉冲响应曲线

In [ ]:
%%stata
graph combine idx_000001_img idx_000016_img idx_000300_img idx_399300_img, ///
xcommon ycommon name(combine_img, replace) scheme(sj)

#### 4.2.2 套利限制

In [ ]:
%%stata -d df_series_ols -force
//时间设定
ge time = _n
tsset time
est clear

//VAR回归
foreach var in total_mv_high total_mv_low  idvol_high idvol_low {
    rename(`var' `var'_s) (return return_s)

    eststo: qui var return img_neg return_s, lags(1/5) exog(month_* weekday_*)
    estadd local Month "Yes", replace
    estadd local Weekday "Yes", replace

    rename(return return_s) (`var' `var'_s)
}

//输出
esttab , keep(return:L*.img_neg) ///
star(* 0.1 ** 0.05 *** 0.01) ///
stats( Month Weekday  r2_1 N, fmt(%3s %3s %12.4f %12.0f)) b(%12.4f) ///
title("Table1 Arbitrage Limit") mtitle("HIGH" "LOW" "HIGH" "LOW")  nogap ///
mgroups("Market Value" "Idiosyncratic Volatility", pattern(1 0 1 0) ) showtabs

## 5. 按照观测窗口构造投资策略(暂时不做)

In [ ]:
def cal_return(df, ma):
    df[f'img_neg_m{ma}'] = (df['img_neg'].rolling(ma).mean())

    # 历史均值K
    df['sell_signal'] = df['img_neg'] >= df[f'img_neg_m{ma}']
    df['sell_signal'] = df['sell_signal'].shift(1)

    # 高于均值投资
    df['img_return'] = np.where(df['sell_signal'], -1*(df['sell_signal']*df['HIGH']), df['shareindex_return'])

    # 去掉空行
    df.dropna(axis=0, inplace=True)

    # 换算
    df['mv_shareindex'] = ((df['shareindex_return'] + 100)/100)
    df['mv_img'] = ((df['img_return'] + 100)/100)
    df['mv_shareindex'] = df['mv_shareindex'].cumprod(axis=0)
    df['mv_img'] = df['mv_img'].cumprod(axis=0)

    return df.rename(columns={'mv_img': f'mv_img_{ma}'})


def start():
    df_in = df_series
    for i in [5, ]:
        df_in = cal_return(df_in, i)

    return df_in

# start()